In [7]:
import set1_util as set1
import set2_util as set2
from Crypto.Cipher import AES
import os
import random
import struct
import base64
import string
KEYSIZE = 16

In [8]:
#Challenge 17 - CBC Padding Oracle
KEY = os.urandom(KEYSIZE)

def fun1():
    a = open('17.txt').read().split('\n')
    index = random.randint(0,9)
    IV = os.urandom(KEYSIZE)
    return AES.new(KEY,AES.MODE_CBC,IV).encrypt(set2.pkcs7(a[index])),IV

def fun2(cipher,iv): #Padding Oracle
    return set2.validate_pkcs(AES.new(KEY,AES.MODE_CBC,iv).decrypt(cipher))

In [9]:
#Exevuting Padding Oracle Attack

cipher,iv = fun1()
fun2(cipher,iv)

blocks = [iv]
temp = ([cipher[i:i+KEYSIZE] for i in xrange(0,len(cipher),KEYSIZE)])
for j in temp:
    blocks.append(j)
# print blocks
# print len(blocks)
for i in xrange(0,len(blocks)):
    index = len(blocks) - i - 1
    print index
    last = blocks[index]
    prev = blocks[index-1]
    for j in xrange(0,256):
        last = last[:15] + chr(j)
        if fun2(last+prev,iv):
            print "Got : ",j
    #Now I need to brute force prev and obtain 01h at end of 

4
Got :  151
3
Got :  129
2
Got :  64
1
Got :  101
0
Got :  250


In [10]:
#Challenge 18 - CTR, The Stream Cipher Mode
class CTR:
    def __init__(self, key, nonce):
        self.key = key
        self.nonce = nonce
        self.ctr = 0

    def keystream(self,byte_count):
        data = struct.pack("<Q",self.nonce)+struct.pack("<Q",byte_count)
        return AES.new(self.key,AES.MODE_ECB).encrypt(data)
    
    def encrypt(self,data):
        #Divide data into 16 bytes block
        data = [data[i:min(i+16,len(data))] for i in range(0,len(data),16)]
        s = ""
        for i in range(0,len(data)):
            s = s + set1.xor_string(data[i].encode('hex'),self.keystream(i).encode('hex'))
        return s.decode('hex')

    def decrypt(self,data):
        return self.encrypt(data)

In [11]:
data = "L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=="
key = "YELLOW SUBMARINE"
nonce = 0
ctr = CTR(key,nonce)
print ctr.decrypt(base64.b64decode(data))

Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


In [12]:
#Scoring Algo #1
#Taken from http://www.counton.org/explorer/codebreaking/frequency-analysis.php
freq = {}
freq['A'] = 8.2
freq['B'] = 1.5
freq['C'] = 2.8
freq['D'] = 4.3
freq['E'] = 12.7
freq['F'] = 2.2
freq['G'] = 2.0
freq['H'] = 6.1
freq['I'] = 7.0
freq['J'] = 0.2
freq['K'] = 0.8
freq['L'] = 4.0
freq['M'] = 2.4
freq['N'] = 6.7
freq['O'] = 7.5
freq['P'] = 1.9
freq['Q'] = 0.1
freq['R'] = 6.0
freq['S'] = 6.3
freq['T'] = 9.1
freq['U'] = 2.8
freq['V'] = 1.0
freq['W'] = 2.4
freq['X'] = 0.2
freq['Y'] = 2.0
freq['Z'] = 0.1

INF = 1e9

def worth(char):
    try:
        k = char.upper()
        return freq[k]
    except:
        if char == ' ': #Space
            return 4.0
        elif char in string.printable: #Other special characters
            return 0.01
        else:
            return -INF

In [13]:
#Scoring Algo #2
#Taken from http://blogs.sas.com/content/iml/2014/09/19/frequency-of-letters.html
freq = {}
freq['A'] = 8.04
freq['B'] = 1.48
freq['C'] = 3.34
freq['D'] = 3.82
freq['E'] = 12.49
freq['F'] = 2.40
freq['G'] = 1.87
freq['H'] = 5.05
freq['I'] = 7.57
freq['J'] = 0.16
freq['K'] = 0.54
freq['L'] = 4.0
freq['M'] = 2.51
freq['N'] = 7.23
freq['O'] = 7.64
freq['P'] = 2.14
freq['Q'] = 0.12
freq['R'] = 6.28
freq['S'] = 6.51
freq['T'] = 9.28
freq['U'] = 2.73
freq['V'] = 1.05
freq['W'] = 1.68
freq['X'] = 0.23
freq['Y'] = 1.66
freq['Z'] = 0.09

INF = 1e9

def worth(char):
    try:
        k = char.upper()
        return freq[k]
    except:
        if char == ' ': #Space
            return 4.0
        elif char in string.printable: #Other special characters
            return 0.01
        else:
            return -INF

In [14]:
#Scoring Algo #3
#Taken from jaybosamiya
freq = {}
freq['a'] = 834
freq['b'] = 154
freq['c'] = 273
freq['d'] = 414
freq['e'] = 1260
freq['f'] = 203
freq['g'] = 192
freq['h'] = 611
freq['i'] = 671
freq['j'] = 23
freq['k'] = 87
freq['l'] = 424
freq['m'] = 253
freq['n'] = 680
freq['o'] = 770
freq['p'] = 166
freq['q'] = 9
freq['r'] = 568
freq['s'] = 611
freq['t'] = 937
freq['u'] = 285
freq['v'] = 106
freq['w'] = 234
freq['x'] = 20
freq['y'] = 204
freq['z'] = 6
freq[' '] = 2320
INF = 1e9

def worth(char):
    try:
        k = char.lower()
        return freq[k]
    except:
        if char in string.printable: #Other special characters
            return 1
        else:
            return -INF

In [79]:
#Challenge 20 - Break Fixed Nonce XOR Statistically
#Divide into cipher, encrypt based on key
a = open('message.txt').read().split('\n')
aa = []
for j in a:
    aa.append(base64.b64encode(j.decode('hex')))
a = aa
# print a
KEY = os.urandom(KEYSIZE)
nonce = random.randint(0,2**64-1)
ctr = CTR(key,nonce)

161
[-]  'Ciphe\x87s cGnabe viEw d E6 a"proximating the a0tHo  oF a proven un)reakAble cipher, th6 ON -time Aad OTP  sCMetimeE known aS t-e Rer mm cDphE '
[-]  'a one\xd8timC 1ad usEsea O ys&ream of completel* Sa doM digit. The  eystReam is combine7 WI1h the Alaintett HIgits Yne at a Tim  tk f!~m tEe C;pher&eEU'
[-]  "the k\x90ystTe m musT 'e C ne ated completely a' Sa doM with at lea8t thE same length a  TH  plainEext anh cMNnot bS used moRe 1haj o oe"
[-]  'this \x98akeU 5he sySt m R ryrcumbersome to imp?eLe t In practice, *nd aS a result the <nE\r1ime paU has nct NEen wiRely used\x0c e=cett (cr tEe M=st c iIH&al   1 IE$ti /S'
[-]  'a str\x90am Ei1her mAk s Q6e =f a much smaller 2nE #orE convenient  ey sUch as 128 bits} bA6ed on Ehis keu, ET geneDates a pSeu!orend!a keTstR7am'
[-]  'a str\x90am Ei1her gEn raP s !uccessive element  NfnthE keystream b*sed On an internal  tAT '
[-]  'in a \x86yncNr.nous St7eaIeci"her a stream of p eTd!-rAndom digits "s geNerated indepen7eNT)y of tYe pla

100.0

In [97]:
#Challenge 19 - Attack using XOR Substitution
#Let us start guessing stuff..
#I have Fixed Byte (unknown) XOR with PlainText (In ASCII Range) => Known Value
def break_CTR(cipher,actual):
    TOTAL = len(cipher)
    MAX_LEN = len(max(cipher,key = len))
    bytes = {}
    #ith byte of each block
    for i in xrange(0,MAX_LEN):
        maxi = 0
        maxj = 1

        #Find first index such that it exists
        for k in xrange(0,TOTAL):
            try:
                fix = ord(cipher[k][i]) 
                break
            except:
                pass

        #Take plaintext of first valid block as j
        for j in xrange(1,128): 
            #Find fixed keystream byte
            fixed = fix ^ j
            flag = True
            score = 0
            for k in xrange(0,TOTAL):
                try:
                    char = chr(ord(cipher[k][i])^fixed)
                    score += worth(char)
                except:
                    pass
            if score>maxi:
#                 print i,maxj
                maxi = score
                maxj = fixed
        bytes[i] = maxj
    bytes[5] = 17^ord('r')^ord('7')
    bytes[9] = 83^ord('A')^ord('g')
    s1 = 'Ciphers canabe viEw d E6 a"proximating the a0tHo  oF a proven un)reakAble cipher, th6 ON -time Aad OTP  sCMetimeE known aS t-e Rer mm cDphE '
    s0 = 'Ciphers can be viewed as approximating the a0tHo  of a proven unbreakable cipher, the ONe-time Aad OTP  sCMetimeE known aS t-e Rer mm cDphE '
    ind = 0
    for x,y in zip(s1,s0):
        bytes[ind] = bytes[ind]^ord(x)^ord(y)
        ind += 1
    # s1 = 
# for j in 
#     bytes[5] = ord('D')
    #Accuracy analysis based on characters differing
    incorrect = 0
    result = []
    for j in xrange(0,TOTAL):
        predicted = ''.join([chr(ord(value)^bytes[index]) for index,value in enumerate(cipher[j])])
#         actual = base64.b64decode(a[j])
#         result.append(predicted)
#         if predicted != actual:
#             print "Incorrectly predicted"
        print "[-] ",repr(predicted)
#             print "[+] ",repr(actual)
#             for x,y in zip(predicted,actual):
#                 incorrect += not(x == y)
    total = sum([len(j) for j in a])
    accuracy = (total - incorrect)*100.0/total
    return accuracy

In [98]:
cipher = []
for j in a:
    cipher.append(ctr.encrypt(base64.b64decode(j)))
break_CTR(cipher,a)

[-]  'Ciphers can be viewed as approximating the a0tHo  of a proven unbreakable cipher, the ONe-time Aad OTP  sCMetimeE known aS t-e Rer mm cDphE '
[-]  'a one-time pad uses a keystream of completel* Sa dom digit. The keystream is combined WIth the Alaintett HIgits Yne at a Tim  tk f!~m tEe C;pher&eEU'
[-]  "the keystream must be generated completely a' Sa dom with at least the same length as THe plainEext anh cMNnot bS used moRe 1haj o oe"
[-]  'this makes the system very cumbersome to imp?eLe t in practice, and as a result the onE\rtime paU has nct NEen wiRely used\x0c e=cett (cr tEe M=st c iIH&al   1 IE$ti /S'
[-]  'a stream cipher makes use of a much smaller 2nE #ore convenient key such as 128 bits. bAsed on Ehis keu, ET geneDates a pSeu!orend!a keTstR7am'
[-]  'a stream cipher generates successive element  Nfnthe keystream based on an internal stATe'
[-]  'in a synchronous stream cipher a stream of p eTd!-random digits is generated independeNTly of tYe plaibteTT and UiphertexT m s

100.0

In [16]:
#Challenge 19 - Break Fixed Nonce XOR
#Divide into cipher, encrypt based on key
a = open('19.txt').read().split('\n')
KEY = os.urandom(KEYSIZE)
nonce = random.randint(0,2**64-1)
ctr = CTR(key,nonce)
cipher = []
for j in a:
    cipher.append(ctr.encrypt(base64.b64decode(j)))
break_CTR(cipher,a)

98.32474226804123

In [11]:
#Challenge 20 - Break Fixed Nonce XOR Statistically
#Divide into cipher, encrypt based on key
a = open('20.txt').read().split(' ')
KEY = os.urandom(KEYSIZE)
nonce = random.randint(0,2**64-1)
ctr = CTR(key,nonce)
cipher = []
for j in a:
    cipher.append(ctr.encrypt(base64.b64decode(j)))
break_CTR(cipher,a)

98.57558139534883

In [ ]:
#Challenge 21 - MT19937 Mersenne Twister RNG
